In [ ]:
pip install fastai


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
from google.colab import files
import io
from fastai.vision.all import *

# Download the Oxford-IIIT Pet Dataset
path = untar_data(URLs.PETS)
path_img = path / 'images'
fnames = get_image_files(path_img)

# Define a function to get labels based on filename
def get_label_from_filename(fname):
    return 0 if fname.name[0].isupper() else 1

# Create a DataBlock for fastai
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=get_label_from_filename,
    item_tfms=Resize(150)
)

dls = dblock.dataloaders(path_img, bs=32)

# Convert fastai DataLoaders to TensorFlow datasets
def tf_dataset_from_dl(dl):
    def generator():
        for batch in dl:
            images, labels = batch
            images = tf.image.convert_image_dtype(images, dtype=tf.float32)  # Ensure images are in float32
            images = tf.transpose(images, perm=[0, 2, 3, 1])  # Transpose to (batch_size, height, width, channels)
            labels = tf.cast(labels, tf.int64)  # Ensure labels are in int64
            for img, lbl in zip(images.numpy(), labels.numpy()):
                yield img, lbl

    return tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(150, 150, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int64)
        )
    )

train_dataset = tf_dataset_from_dl(dls.train)
val_dataset = tf_dataset_from_dl(dls.valid)

# Apply data augmentation
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    # Randomly zoom by cropping and resizing
    image = tf.image.random_crop(image, size=[150, 150, 3])
    return image, label

train_dataset = train_dataset.map(augment).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Debug the dataset
for image, label in train_dataset.take(1):
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)

# Define a more complex CNN Model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the Model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the Model
history = model.fit(
    train_dataset,
    epochs=15,
    validation_data=val_dataset
)

# Evaluate the Model
loss, accuracy = model.evaluate(val_dataset)
print(f'Validation accuracy: {accuracy * 100:.2f}%')





Image shape: (32, 150, 150, 3)
Label shape: (32,)
Epoch 1/15
184/184 [==============================] - 436s 2s/step - loss: 0.6024 - accuracy: 0.6788 - val_loss: 0.5760 - val_accuracy: 0.6881
Epoch 2/15
184/184 [==============================] - 436s 2s/step - loss: 0.5753 - accuracy: 0.6979 - val_loss: 0.5470 - val_accuracy: 0.7456
Epoch 3/15
184/184 [==============================] - 429s 2s/step - loss: 0.5525 - accuracy: 0.7220 - val_loss: 0.5172 - val_accuracy: 0.7659
Epoch 4/15
184/184 [==============================] - 432s 2s/step - loss: 0.5374 - accuracy: 0.7339 - val_loss: 0.5175 - val_accuracy: 0.7463
Epoch 5/15
184/184 [==============================] - 434s 2s/step - loss: 0.5227 - accuracy: 0.7412 - val_loss: 0.5011 - val_accuracy: 0.7131
Epoch 6/15
184/184 [==============================] - 431s 2s/step - loss: 0.4910 - accuracy: 0.7678 - val_loss: 0.4591 - val_accuracy: 0.7984
Epoch 7/15
184/184 [==============================] - 429s 2s/step - loss: 0.4762 - accuracy